In [1]:
%%writefile Analysis_Funs.py

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats as st
import statsmodels.api as sm
import QUANTAXIS as QA

from sklearn import linear_model

import numba as nb

def VIF(df):
    """方差膨胀法共线性校验：
        :df：{pd.DataFrame}
        :return：{pd.DataFrame} -- 各变量描述，VIF>10则共线可能
    """
    from statsmodels.stats.outliers_influence import variance_inflation_factor 
    tmp_df = df.assign(const=1)
    x = np.array(df)
    vif_list = [variance_inflation_factor(x, i) for i in range(x.shape[1])]
    df_vif = pd.DataFrame({'variable': list(tmp_df.columns), 'vif': vif_list})
    df_vif = df_vif[~(df_vif['variable'] == 'const')]   # 删除常数项
    return df_vif

def get_LR_params_fast(x_array,y_array):
    """快速最小二乘：
        :param x_array：{list or np.array}
        :param y_array：{list or np.array}
        :return：{list} -- [-1]为截距，其他为系数
    """
    if not isinstance(x_array, np.ndarray):
        x= np.array(x_array)
    else:
        x=x_array
        
    if len(x.shape) < 2:
        x=x.reshape(-1,1)
    x = np.hstack([x,np.ones([x.shape[0],1])])
    params = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y_array)
    return params

def get_rank_ic(factor_standardized, ret_forward):
    """
       计算因子的信息系数
        :param factor_standardized: {pd.Series:multiIndex[date,code]} --标准化后的因子值
        :param ret_forward: {pd.Series:multiIndex[date,code]} --下一期的股票收益率
        :return {pd.Series}
        !注意factor_standardized必须是pd.Series
    """
    
    assert len(factor_standardized.index.get_level_values(1).unique()) > 4, '股票数量必须大于4，否则没啥意义啊'

    #index取交集
    common_index = factor_standardized.index.get_level_values(0).unique().intersection(ret_forward.index.get_level_values(0).unique())
    ic_data = pd.Series([None]*len(common_index),index=common_index)
    
    df = pd.DataFrame({'factor_standardized':factor_standardized.loc[common_index], 'ret_f':ret_forward})
    df = df[(~pd.isnull(df['factor_standardized'])) & (~pd.isnull(df['ret_f']))]
    

    # 计算相关系数
    for dt in df.index.get_level_values(0).unique():
        if len(df.loc[dt]) < 5:
            #'参与计算标的小于5只时，跳过'
            continue
        ic = df['factor_standardized'][dt].rank().corr(df['ret_f'][dt].rank(),method='pearson')
        ic_data[dt] = ic
        
    return ic_data

def get_ic_desc(ic_data):
    """因子信息系数的描述
       :param ic_data:{pd.Series，Index[date,]} --rankIC值, 
       :return: {tuple(mean,std,t_static,p_value)}
    """
    tmp = ic_data.dropna()
    mean = tmp.mean()
    std = tmp.std()
    t_static,p_value = st.ttest_ind(tmp, [0] * len(tmp))
    return mean,std,t_static,p_value
    
def get_ic_ir(ic_data):
    return ic_data.mean()/ic_data.std()

def auto_describe(df):
    report = sv.analyze(df,pairwise_analysis='auto')
    report.show_notebook()
    
def get_winning_rate(ic_data_df):
    """因子胜率
       :param ic_data_df:{pd.DataFrame，Index[date,]} --rankIC值, 
       :return: {pd.DataFrame}
    """
    df_ = ic_data_df.dropna()
    ic_mean_sign = np.sign(df_.mean())
    return (np.sign(df_) == ic_mean_sign).sum()/df_.count()

def climbing(data:np.ndarray):
    '''上升
       :param data: ;
       :return：{tuple} -- tuple(最大上升,上升率,最大上升周期,底下标,顶下标)
    '''
    top_index = np.argmax(data-np.minimum.accumulate(data))
    if top_index == 0:
        return 0, 0, 0, 0, 0
    bottom_index = np.argmin(data[:top_index])
    
    duration = bottom_index - top_index
    up = data[bottom_index] - data[top_index]
    up_rate = np.round(up / data[bottom_index], 2)
    return up, up_rate, duration, bottom_index, top_index


def retracing(data:np.ndarray):
    '''回撤
       :param data: ;
       :return：{tuple} -- tuple(最大回撤,回撤率,最大回撤周期,顶下标,底下标)
    '''
    bottom_index = np.argmax(np.maximum.accumulate(data)-data)
    if bottom_index == 0:
        return 0, 0, 0, 0, 0
    top_index = np.argmax(data[:bottom_index])
    
    retrace_duration = bottom_index - top_index
    retrace = data[top_index] - data[bottom_index]
    retrace_rate = np.round(retrace / data[top_index], 2)
    return retrace, retrace_rate, retrace_duration, top_index, bottom_index

def cross_sign(direct_signals:np.ndarray):
    """根据数据方向序列，取得驻点
    """
    ## 把0 bfill掉，避免0的时候误认为拐点
    mask0 = direct_signals==0
    idx0 = np.where(~mask0,np.arange(mask0.shape[0]),0)
    np.maximum.accumulate(idx0, out=idx0)
    signal = direct_signals[idx0]
    
    dif_shift_neg1 = np.append(signal[1:],0)
    cross_sign = np.where(signal==dif_shift_neg1,0,np.where(signal<dif_shift_neg1,1,-1))
    cross_sign[0]=0
    cross_sign[-1]=0
    return cross_sign

def get_direct_sign(arr:np.ndarray):
    """取得序列变化方向
    """
    direct_sign = np.append(0,np.sign(np.diff(arr)))
    return direct_sign

def get_cross_sign(direct_signals:np.ndarray):
    """根据数据方向序列，取得驻点
    """
    ## 把0 bfill掉，避免0的时候误认为拐点
    mask0 = direct_signals==0
    idx0 = np.where(~mask0,np.arange(mask0.shape[0]),0)
    np.maximum.accumulate(idx0, out=idx0)
    signal = direct_signals[idx0]
    
    dif_shift_neg1 = np.append(signal[1:],0)
    cross_sign = np.where(signal==dif_shift_neg1,0,np.where(signal<dif_shift_neg1,1,-1))
    cross_sign[0]=0
    cross_sign[-1]=0
    return cross_sign


def get_longest_recover_duration(data:np.ndarray, cross_sign_arr:np.ndarray):
    '''计算恢复原点位所用的周期
       :param cross_sign_arr: --拐点信号序列，1上折，-1下折，0保持;
       :return：{tuple} -- tuple(最大周期,起点下标,终点下标)
    '''
    cross_down_idx = np.argwhere( cross_sign_arr==-1 )  # 下折点统计
    max_recover_duration = 0 #默认为无回复周期
    max_recover_start = None
    max_recover_end = None
    # 查找整个窗口内的最大回复周期，既从下折开始，恢复到原始价格所用的时间窗口
    if len(cross_down_idx) > 0:
        for cd_idx in cross_down_idx.T[0]:
            #遍历下折点
            cross_value = data[cd_idx]
            #从下折点的下一个点开始找回复点
            recovers = np.argwhere(data[cd_idx+1:]>=cross_value)

            if len(recovers)>0:
                # 第一个恢复点的下标，相当于本次下折的最小回复周期
                # 由于下标起点为0，所以实际周期+1,这个周期不包括起点本身
                recover_duration = recovers.T[0][0]+1 
                # 更新最大周期
                if recover_duration > max_recover_duration:
                    max_recover_duration = recover_duration
                    max_recover_start = cd_idx #本身为起点
                    max_recover_end = cd_idx+recover_duration
    return max_recover_duration, max_recover_start, max_recover_end


def get_longest_rollercoaster_duration(data:np.ndarray, cross_sign_arr:np.ndarray):
    '''计算最大过山车周期
       :param cross_sign_arr: --拐点信号序列，1上折，-1下折，0保持;
       :return：{tuple} -- tuple(最大周期,起点下标,终点下标)
    '''
    cross_up_idx = np.argwhere( cross_sign_arr==1 )  # 上折点统计
    max_rollercoaster_duration = 0 #默认为无回复周期
    max_rollercoaster_start = None
    max_rollercoaster_end = None
    # 查找整个窗口内的最大回复周期，既从上折开始，恢复到原始价格所用的时间窗口
    if len(cross_up_idx) > 0:
        #遍历上折点
        for cu_idx in cross_up_idx.T[0]:
            cross_value = data[cu_idx]
            #从上折点的下一个点开始找回归点
            regorge = np.argwhere(data[cu_idx+1:]<=cross_value)
            if len(regorge)>0:
                # 第一个恢复点的下标，相当于本次下折的最小回复周期
                # 由于下标起点为0，所以实际周期+1,这个周期不包括起点本身
                rollercoaster_duration = regorge.T[0][0]+1 
                # 更新最大周期
                if rollercoaster_duration > max_rollercoaster_duration:
                    max_rollercoaster_duration = rollercoaster_duration
                    max_rollercoaster_start = cu_idx #本身为起点
                    max_rollercoaster_end = cu_idx + rollercoaster_duration
    return max_rollercoaster_duration, max_rollercoaster_start,max_rollercoaster_end


@nb.jit(nopython=True)
def ma_fast(data:np.array, window:int):
    '''
        快速MA,数据中禁止存在nan
        不考虑jit的话，可以使用下面的strided_rolling
        例：ma_fast(x[~np.isnan(x)],window)
    '''
    assert np.isnan(data).any()==False ,'can not contain nan'
    sum_ = 0
    result = np.full(len(data),np.nan)
    
    for i in range(0, len(data)):
        sum_ += data[i]
        if i > window-1:
            sum_ -= data[i-window]
        if i >= window-1:
            result[i] = sum_ / window
    return result

@nb.jit(nopython=True)
def inv_num_fast(series):
    '''
        计算逆序数个数
    '''
    count = 0
    for i in range(len(series)):
        x = series[i]
        for j in range(i):
            if x < series[j]:
                count+=1
    return count
    
@nb.jit(nopython=True)
def ma_power_np_base(data, ma_param=np.array(range(5, 30))):
    '''
        MA均线多头排列能量强度
        :param data:{nb.typed.List} --价格或指标序列;
        :param ma_param:{np.array} --移动平均参数;
        :return：{list} [0-1] 
    '''
    assert len(ma_param) > 1, 'size of ma_param must > 1'
    ma_np = np.empty((len(data), len(ma_param)))
    ma_count = 0

    for r in ma_param:
        sum = 0
        for i in range(0, len(data)):
            sum += data[i]
            if i > r-1:
                sum -= data[i-r]
            if i >= r-1:
                ma_np[i, ma_count]=sum/r
        ma_count += 1

    ma_max = max(ma_param)
    param_len = len(ma_param)
    param_count = (param_len * (param_len - 1)) / 2
    
    num = np.zeros(len(data))
    ratio = np.zeros(len(data))
#     with np.errstate(invalid='ignore', divide='ignore'):
    for i in range(ma_max,len(data)):
        # 排在数组越靠前，周期就小，
        # 在多头趋势中，理论上应该最快反应，也就是值越大。所以逆序能代表其强度
        num[i] = inv_num_fast(ma_np[i, :]) 
        ratio[i] = num[i] / param_count

    return ratio

def ma_power_np(data_series, ma_param=range(5, 30)):
    '''
        MA均线多头排列能量强度
        :param data_series:{pd.Series} --价格或指标序列;
        :param ma_param:{list} --移动平均参数;
        :return：{pd.Series} [0-1] 
    '''
    param = np.array(ma_param)
    res = ma_power_np_base(nb.typed.List(data_series.to_list()), param)
    return pd.Series(res,index=data_series.index)


@nb.jit(nopython=True)
def peak_detection_z(y, lag = 5, std_times = 3, convolute_coef = 0.5):
    """
        Robust peak detection algorithm (using z-scores)
        自带鲁棒性极值点识别，利用方差和ZSCORE进行时间序列极值检测。算法源自：
        https://stackoverflow.com/questions/22583391/
        本实现使用Numba JIT优化，比原版（上面）大约快了500倍。
        :param y:{np.array | nb.typed.List} --数据;
        :param std_times:{float} --偏差几倍于标准差;
        :param convolute_coef:{float}[0-1] --越过标准差边界时，
                下一论被纳入计算的当前y，与y_t-1加权平均;既系数越小，状态的持续性越强。
        :return：{list[3,]}  --list[0]:[-1,0,1]点状态；list[1]:均值序列；list[2]:标准差序列；
    """
    ret_signals = np.zeros((3, len(y),))
    idx_signals = 0
    idx_avgFilter = 1
    idx_stdFilter = 2

    filteredY = np.copy(y)
    ret_signals[idx_avgFilter, lag - 1] = np.mean(y[0:lag])
    ret_signals[idx_stdFilter, lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - ret_signals[idx_avgFilter, i - 1]) > std_times * ret_signals[idx_stdFilter, i - 1]:
            if y[i] > ret_signals[idx_avgFilter, i - 1]:
                ret_signals[idx_signals, i] = 1
            else:
                ret_signals[idx_signals, i] = -1

            filteredY[i] = convolute_coef * y[i] + (1 - convolute_coef) * filteredY[i - 1]
            ret_signals[idx_avgFilter, i] = np.mean(filteredY[(i - lag):i])
            ret_signals[idx_stdFilter, i] = np.std(filteredY[(i - lag):i])
        else:
            ret_signals[idx_signals, i] = 0
            filteredY[i] = y[i]
            ret_signals[idx_avgFilter, i] = np.mean(filteredY[(i - lag):i])
            ret_signals[idx_stdFilter, i] = np.std(filteredY[(i - lag):i])

    return ret_signals

def peak_detection(data_series, lag = 5, std_times = 3, convolute_coef = 0.5):
    """
        Robust peak detection algorithm (using z-scores)
        :param y:{list|np.array|pd.Series} --数据;
        :param std_times:{float} --偏差几倍于标准差;
        :param convolute_coef:{float}[0-1] --越过标准差边界时，
                下一论被纳入计算的当前y，与y_t-1加权平均;既系数越小，状态的持续性越强。
        :return：{list[3,]}  --list[0]:[-1,0,1]点状态；list[1]:均值序列；list[2]:标准差序列；
    """
    ret_signals = peak_detection_z(np.array(data_series), lag = lag, std_times = std_times, convolute_coef = convolute_coef)
    if isinstance(data_series, pd.Series):
        return pd.Series(ret_signals[0],index=data_series.index)
    else:
        return pd.Series(ret_signals[0])


def strided_rolling(a, L, S):  
    '''
    Pandas rolling for numpy
    # Window len = L, Stride len/stepsize = S
    '''
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S * n,n))

def rolling_poly(data_arr:np.ndarray, window:int=252, ploy_vars_count=9) -> np.ndarray:
    '''
    一元九项式滚动分解拟合
    '''
    x_index = range(window)
    def last_poly(sx):
        p = np.polynomial.Chebyshev.fit(x_index, sx, ploy_vars_count)
        return p(x_index)[-1]

    if (len(data_arr) > window):
        x = strided_rolling(data_arr, window, 1)
        return np.r_[np.full(window - 1, np.nan), 
                     np.array(list(map(last_poly, x)))]
    else:
        x_index = range(len(data_arr))
        p = np.polynomial.Chebyshev.fit(x_index, data_arr, ploy_vars_count)
        y_fit_n = p(x_index)
        return y_fit_n

def poly_effectiveness(data_arr:np.ndarray, window:int=252, ploy_vars_count:int=5, get_fitted:bool=False):
    '''
        多项式滚动拟合相关性 
        --经验值：0.5左右及以下为随机序列。真实价格序列>0.8，另外且方差下降，也说明了波动在增大
        :param data_arr:{np.ndarray} 
        :param window:{int}  --回望周期
        :param ploy_vars_count:{int}  --多项式项数,5时，各种随机生成算法的结果约0.5
        :param get_fitted:{bool} --True时返回拟合数据。 
        :return：{float[0-1]|tuple(folat,np.ndarray)} 
    '''
    fitted = rolling_poly(data_arr, window, ploy_vars_count)
    cor = np.corrcoef(data_arr[window-1:],fitted[window-1:])
    if get_fitted:
        return cor[0][1],fitted
    return cor[0][1]


def RPS_rank_backward(stock_df, deal_cloumn='close', ret_stride=5):
    """
        来源：RPS英文全称Relative Price Strength Rating，即股价相对强度，
        该指标是欧奈尔CANSLIM选股法则中的趋势分析，具有很强的实战指导意义。
        RPS指标是指在一段时间内，个股涨幅在全部股票涨幅排名中的位次值。
        
        这个函数计算每个交易日所有收益排行的《反序》，既收益越大，排名的数值越大。
        :param stock_df:{np.DataFrame} 
        :param deal_cloumn:{str}  --所要处理的字段
        :param ret_stride:{int}  --回报周期
        :return：{np.DataFrame} 
    """
    rets = smpl.get_current_return(stock_df,deal_cloumn,ret_stride)
    rets.name = deal_cloumn+'_pct'
    
    def get_RPS(series):
        """计算RPS(时间截面)
        """
        ret_rank = pd.DataFrame(series)
        ret_rank['rank'] = series.rank(ascending=True, pct=True)
        return ret_rank
    
    ret_indices = pd.DataFrame(rets)
#     ret_indices[deal_cloumn] = df[deal_cloumn]
    ret_indices['rank'] = np.nan
    ret_indices.loc[:, ['rank']] = rets.groupby(level=[0]).apply(lambda x:get_RPS(x))
    return ret_indices


@nb.jit(nopython=True)
def LIS(X:np.array) -> tuple:
    """计算最长递增子序列
       :return: {tuple(list(序列值), list(序列idx))}
    """
    N = len(X)
    P = [0] * N
    M = [0] * (N + 1)
    L = 0
    for i in range(N):
        lo = 1
        hi = L
        while lo <= hi:
            mid = (lo + hi) // 2
            if (X[M[mid]] < X[i]):
                lo = mid + 1
            else:
                hi = mid - 1

        newL = lo
        P[i] = M[newL - 1]
        M[newL] = i

        if (newL > L):
            L = newL

    S = []
    pos = []
    k = M[L]
    for i in range(L - 1, -1, -1):
        S.append(X[k])
        pos.append(k)
        k = P[k]
    return S[::-1], pos[::-1]


@nb.jit(nopython=True)
def LDS(X:np.array) -> tuple:
    """计算最长递减子序列 Longest decreasing subsequence
       :return: {tuple(list(序列值), list(序列idx))}
    """
    N = len(X)
    P = [0] * N
    M = [0] * (N + 1)
    L = 0
    for i in range(N):
        lo = 1
        hi = L
        while lo <= hi:
            mid = (lo + hi) // 2
            if (X[M[mid]] > X[i]):
                lo = mid + 1
            else:
                hi = mid - 1

        newL = lo
        P[i] = M[newL - 1]
        M[newL] = i

        if (newL > L):
            L = newL

    S = []
    pos = []
    k = M[L]
    for i in range(L - 1, -1, -1):
        S.append(X[k])
        pos.append(k)
        k = P[k]
    return S[::-1], pos[::-1]


@nb.jit(nopython=True)
def timeline_event_integral(signals:np.array) -> np.array:
    """
        事件发生的连续性计数
        计算时域金叉/死叉信号的累积卷积和(死叉(1-->0)清零)，
        经测试for实现最快，比reduce快
        :param signals:{np.array} [0|1]  
                -- jx = timeline_integral(np.where(data.diff(1) > 0, 1,0))
                   sx = timeline_integral(np.where(data.diff(1) < 0, 1,0))
        :return: {np.array}
    """
    t = np.zeros(len(signals)).astype(np.int32)
    for i, s in enumerate(signals):
        t[i] = s * (t[i - 1] + s)
    return t


@nb.jit(nopython=True)
def timeline_event_duration(signals:np.array) -> np.array:
    """
        事件发生的《间隔》,既：无事发生的连续性计数
        计算时域金叉/死叉信号的累积卷积和(死叉(1-->0)不清零，金叉(0-->1)清零)
        经测试for最快，比reduce快(无jit，jit的话for就更快了)
        :param signals:{np.array[0|1] } 
        :return: {np.array}
    """
    t = np.zeros(len(signals)).astype(np.int32)
    for i, s in enumerate(signals):
        t[i] = (t[i - 1] + 1) if (s != 1) else 0
    return t

def timeline_event_continuity(directions):
    """
    计算事件发生的连续性计数，含方向
    !!!!注意，0不计入事件。
        不要直接简单传递diff差值，无法直接处理差分为0的情况。
        传递sign更合理
    
    :param tm:{np.array[-1|0|1]} 
    :return: {np.array}
    """
    with np.errstate(invalid='ignore', divide='ignore'):
        trend_jx = timeline_event_integral(np.where(directions > 0, 1, 0))
        trend_sx = np.sign(directions) * timeline_event_integral(np.where(directions < 0, 1, 0))
    return np.array(trend_jx + trend_sx).astype(np.int32)

def feature_JXSX_timeline(serise):
    """
    技术指标特征的金叉死叉时序
    !!!!注意，不支持输入0，既无事件的间隔
        0会被认为是事件翻转
    :param tm:{np.array[-1|1]} 
    :return: {np.array}
    实际使用上方的timeline_event_continuity更好，支持0跳过。
    """
    features_jx_before = timeline_event_duration(np.where(serise > serise.shift(1), 1, 0))
    features_sx_before = timeline_event_duration(np.where(serise < serise.shift(1), 1, 0)) 
    return timeline_event_continuity(np.where((features_jx_before < features_sx_before), 1, -1))



Overwriting Analysis_Funs.py


In [ ]:
######未投入使用方法

# 凯利
# w = return/(variance*normalizer)

%%time
# from PyEMD import EEMD,EMD
# import numpy as np

max_imf = 17
S = tmp.values[100:150] ## <---------------#####
T = range(len(S))

# EEMD计算
eemd = EEMD()
eemd.trials = 50
eemd.noise_seed(12345)

E_IMFs = eemd.eemd(S, T, max_imf)
imfNo = E_IMFs.shape[0]
tMin, tMax = np.min(T), np.max(T)

# Plot results in a grid
c = int(np.floor(np.sqrt(imfNo + 1)))
r = int(np.ceil((imfNo + 1) / c))

fig = plt.figure(figsize=(640/72,50/72))
plt.plot(T, S, 'r')
plt.xlim((tMin, tMax))
plt.title("xxxxxxxxxxxxxxxxxxx")
plt.show()

for num in range(imfNo):
    fig = plt.figure(figsize=(640/72,50/72))
    plt.plot(T, E_IMFs[num], 'g')
    plt.xlim((tMin, tMax))
    plt.title("Imf " + str(num + 1))
    plt.show()
plt.show()

def PSY(price, period=12, ma=6):
    """
    心理线（PSY）指标是将一定时期内投资者趋向买方或卖方的心理事实转化为数值，从而判断股价的未来趋势。
    它对股市短期走势的研判具有一定的参考意义。心理线（PSY）指标将一定时期内投资者趋向买方或卖方的心理事实转化为数值，从而判断股价的未来趋势。

    1.PSY=(N日内上涨天数/N)*100
    2.PSYMA=PSY的M日简单移动平均
    3.参数N设置为12日，参数M设置为6日
    """
    diff = price.diff().fillna(0)
    diff_direct = np.where(diff > 0, 1, 0)
    psy = np.zeros((len(price),))
    psy[:period] *= np.nan
    for i in range(period, len(price)):
        psy[i] = (diff_direct[i-period+1:i+1].sum()) / period
        
    slices = strided_rolling(psy.copy(), ma, 1)
    idx = 0
    for s in slices:
        psy[ma-1+idx] = s.sum()/ma
        idx+=1

    return psy*100

In [17]:
######待参考方法
def mapower_trend(series,features:pd.DataFrame=None,verbose:bool=False,):

    ma_power = ma_power_np(series, range_list=np.array(range(5, 30)))
    features['ma_power30'] = ma_power
    features['ma_power30_continuity'] = timeline_event_continuity(np.where((features['ma_power30'].diff() > 0) | \
                                                                        (features['ma_power30'] > 0.999), 1 ,-1))

    features['ma_power30_CHECKOUT_LOW'] = np.where(features['ma_power30_continuity'] == 1, 
                                                              features['ma_power30'].shift(1), 
                                                              np.nan)
    features['ma_power30_CHECKOUT_LOW'] = features['ma_power30_CHECKOUT_LOW'].ffill()
    features['ma_power30_CHECKOUT_HIGH'] = np.where(features['ma_power30_continuity'] == -1, 
                                                              features['ma_power30'].shift(1), 
                                                              np.nan)
    features['ma_power30_CHECKOUT_HIGH'] = features['ma_power30_CHECKOUT_HIGH'].ffill()

    features['ma_power30_PEAK_LOW_BEFORE'] = timeline_event_duration(np.where((features['ma_power30'].shift(1) < 0.001) & \
                                                                            (features['ma_power30'] > 0.01), 1, 
                                                                            np.where((features['ma_power30'].shift(1) < features['ma_power30']) & \
                                                                                    (features['ma_power30'] < 0.0168), 1, 0)))
    features['ma_power30_PEAK_HIGH_BEFORE'] = timeline_event_duration(np.where((features['ma_power30'].shift(1) > 0.999) & \
                                                                            (features['ma_power30'] < 0.99), 1, 
                                                                            np.where((features['ma_power30'].shift(1) > features['ma_power30']) & \
                                                                                    (features['ma_power30'] > 0.982), 1, 0)))    

    return features

In [250]:
import QUANTAXIS as QA
import base.JuUnits as u
from QUANTAXIS.QAUtil.QAParameter import MARKET_TYPE, RUNNING_ENVIRONMENT, ORDER_DIRECTION
data = QA.QA_quotation(['000001','000002', '000004', '000005', '000006'], '2010-12-05', '2020-12-31', source=QA.DATASOURCE.MONGO,
                               frequence='day', market=MARKET_TYPE.STOCK_CN, 
                               output=QA.OUTPUT_FORMAT.DATASTRUCT)
data.data

open   high    low  close    volume        amount
date       code                                                      
2010-12-06 000001  16.60  16.88  16.50  16.74  217923.0  3.642054e+08
           000002   8.51   8.69   8.45   8.49  982345.0  8.443094e+08
           000004  12.57  12.57  12.13  12.32    7599.0  9.389200e+06
           000005   3.89   3.93   3.87   3.89   63534.0  2.476463e+07
           000006   7.96   8.37   7.80   8.25  511592.0  4.185568e+08
...                  ...    ...    ...    ...       ...           ...
2020-12-31 000001  19.21  19.58  19.02  19.34  924503.0  1.781736e+09
           000002  28.29  28.80  28.24  28.70  651991.0  1.862538e+09
           000004  20.69  21.09  20.31  20.70   22304.0  4.619703e+07
           000005   2.50   2.53   2.50   2.53   67806.0  1.701545e+07
           000006   5.55   5.68   5.52   5.61   69448.0  3.895084e+07

[11407 rows x 6 columns]

In [280]:
# import Pretreat_Tools as pretreat
# import Sample_Tools as smpl

# factor_standardized_df = pd.DataFrame(pretreat.standardize(data.close, multi_code=True))
# ret_forward = smpl.get_forward_return(data.data,'close')

# ic = factor_standardized_df.apply(lambda x: get_rank_ic(x, forward_return),axis=0)
# ic_mean = ic.rolling(12).mean()
# ic_mean = ic_mean.shift(1)
# weight = ic_mean.apply(lambda x: x / x.sum(), axis=1)
# factor_weighted = FactorWeight.weighted(factor_standardized_df, weight)
factor_weighted
ic_mean

,close
date,
2019-12-25,NaN
2019-12-26,NaN
2019-12-27,NaN
2019-12-30,NaN
2019-12-31,NaN
...,...
2020-12-25,-5.833333e-02
2020-12-28,8.333333e-03
2020-12-29,4.857226e-16


In [281]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(ic_mean)

                   close
date                    
2019-12-25           NaN
2019-12-26           NaN
2019-12-27           NaN
2019-12-30           NaN
2019-12-31           NaN
2020-01-02           NaN
2020-01-03           NaN
2020-01-06           NaN
2020-01-07           NaN
2020-01-08           NaN
2020-01-09           NaN
2020-01-10           NaN
2020-01-13 -4.166667e-02
2020-01-14 -1.500000e-01
2020-01-15 -2.166667e-01
2020-01-16 -1.500000e-01
2020-01-17 -1.250000e-01
2020-01-20 -1.916667e-01
2020-01-21 -1.333333e-01
2020-01-22 -8.333333e-03
2020-01-23 -4.166667e-02
2020-02-03 -4.625929e-18
2020-02-04  5.833333e-02
2020-02-05  5.000000e-02
2020-02-06 -1.666667e-02
2020-02-07 -8.333333e-03
2020-02-10  7.500000e-02
2020-02-11  8.333333e-03
2020-02-12 -5.833333e-02
2020-02-13  1.666667e-02
2020-02-14  5.000000e-02
2020-02-17 -6.666667e-02
2020-02-18 -7.500000e-02
2020-02-19 -1.666667e-01
2020-02-20 -2.166667e-01
2020-02-21 -1.916667e-01
2020-02-24 -9.166667e-02
2020-02-25  4.166667e-02


In [52]:
##############l拉格朗日
 
#目标函数：
def func(args):
    fun = lambda x: 60 - 10*x[0] - 4*x[1] + x[0]**2 + x[1]**2 - x[0]*x[1]
    #fun = lambda x: 10 - x[0]**2 - x[1]**2
    return fun
 
#约束条件，包括等式约束和不等式约束
def con(args):
    cons = [{'type': 'eq', 'fun': lambda x: x[0]+x[1]-8}]
    #cons = [{'type': 'ineq', 'fun': lambda x: x[1]-x[0]**2},
    #        {'type': 'eq', 'fun': lambda x: x[0]+x[1]}]
    return cons 
 

args = ()
args1 = ()
cons = con(args1)
x0 = np.array((2.0, 1.0))  #设置初始值，初始值的设置很重要，很容易收敛到另外的极值点中，建议多试几个值

#求解#
res = minimize(func(args), x0, method='SLSQP', constraints=cons)
print(res.success)
print("x1=",res.x[0],";  x2=",res.x[1])
print("最优解为：",res.fun)


True
x1= 4.999999943481969 ;  x2= 3.000000056518032
最优解为： 17.000000000000007


In [2]:
import QUANTAXIS as QA
import base.JuUnits as u
from QUANTAXIS.QAUtil.QAParameter import MARKET_TYPE, RUNNING_ENVIRONMENT, ORDER_DIRECTION
data = QA.QA_quotation(['000001','000002', '000004', '000005', '000006'], '2020-12-25', '2020-12-31', source=QA.DATASOURCE.MONGO,
                               frequence='day', market=MARKET_TYPE.STOCK_CN, 
                               output=QA.OUTPUT_FORMAT.DATASTRUCT)
data.data

open   high    low  close     volume        amount
date       code                                                       
2020-12-25 000001  18.26  18.26  17.80  18.04   577077.0  1.038128e+09
           000002  27.95  28.22  27.60  28.10   425877.0  1.190254e+09
           000004  21.19  21.98  21.16  21.71    21984.0  4.779672e+07
           000005   2.50   2.53   2.49   2.52    75213.0  1.887256e+07
           000006   5.65   5.74   5.63   5.65    67918.0  3.864496e+07
2020-12-28 000001  18.02  18.86  17.96  18.85  1270337.0  2.352947e+09
           000002  28.18  28.65  27.86  28.42   583680.0  1.656263e+09
           000004  21.01  21.49  20.40  20.45    36042.0  7.497581e+07
           000005   2.51   2.51   2.48   2.49    67099.0  1.674974e+07
           000006   5.62   5.65   5.48   5.50    70810.0  3.915675e+07
2020-12-29 000001  18.87  19.30  18.70  19.17   963092.0  1.837947e+09
           000002  28.56  28.86  28.32  28.52   539042.0  1.544401e+09
           000004  20.59  21.28  20.40  20.66    29657.0  6.186338e+07
           000005   2.50   2.51   2.48   2.50    40798.0  1.018245e+07
           000006   5.50   5.65   5.48   5.53    55795.0  3.104783e+07
2020-12-30 000001  19.00  19.20  18.72  19.20   978497.0  1.854083e+09
           000002  28.47  28.65  28.22  28.38   499718.0  1.417604e+09
           000004  20.98  20.98  20.31  20.47    19584.0  4.022028e+07
           000005   2.50   2.52   2.49   2.49    73658.0  1.843081e+07
           000006   5.51   5.57   5.47   5.54    46516.0  2.571174e+07
2020-12-31 000001  19.21  19.58  19.02  19.34   924503.0  1.781736e+09
           000002  28.29  28.80  28.24  28.70   651991.0  1.862538e+09
           000004  20.69  21.09  20.31  20.70    22304.0  4.619703e+07
           000005   2.50   2.53   2.50   2.53    67806.0  1.701545e+07
           000006   5.55   5.68   5.52   5.61    69448.0  3.895084e+07

In [204]:
%%time
forward_return = data.close.groupby(level=1, group_keys=False).apply(lambda x:((x-x.shift(1))/x.shift(1)).shift(-1))
test_factor = data.volume
test_factor = pd.DataFrame({"a":test_factor, "b":data.close, "c":np.sqrt(data.close)+np.log(test_factor)})
# data['close']

# 标准化
def standardize(data, multi_code=False):
    if multi_code:
        return data.groupby(level=1, group_keys=False).apply(lambda x: standardize(x,multi_code=False))
    else:
        return (data - data.mean())/data.std()



test_factor_stda = standardize(test_factor,multi_code=True)
print(test_factor_stda)



# print("mean,std,t,tt,p:",get_ic_desc(ic_datas))
# print('ir',get_ic_ir(ic_datas))

# test_df = pd.DataFrame({"a":test_factor, "b":test_factor*2})
# print(test_df)
# # print(FactorWeight.equal_weight(test_df))
# print(test_df.columns)


# # equal_weight 
# weight = pd.Series([1. / len(test_factor_stda.columns)] * len(test_factor_stda.columns),index=test_factor_stda.columns)
# print(weight)
# print(test_factor_stda.apply(lambda x: (x*weight).sum(),axis=1))

# print(FactorWeight.equal_weight(test_factor_stda))


# ic_datas = test_factor_stda.apply(lambda x: get_rank_ic(x, forward_return),axis=0)
# print(ic_datas)
# ic_datas_mean = ic_datas.rolling(12).mean().shift(1)
# ic_datas_mean = ic_datas_mean.shift(1)
# # print(ic_datas_mean)
# weight = ic_datas_mean.apply(lambda x: x / x.sum(), axis=1)
# print(weight)
# print(test_factor_stda.apply(lambda x: (x*weight).sum(),axis=1))

# print(FactorWeight.ic_weight(test_factor_stda, forward_return))


# ic_ir_datas = ic_datas.rolling(12).mean()/ ic_datas.rolling(12).std()
# ic_ir_datas = ic_ir_datas.shift(1)
# print(ic_ir_datas)
# weight = ic_ir_datas.apply(lambda x: x / x.sum(), axis=1)
# print(weight)
# print(test_factor_stda.apply(lambda x: (x*weight).sum(),axis=1))

# print(FactorWeight.ic_ir_weight(test_factor_stda, forward_return))


                          a         b         c
date       code                                
2019-12-25 000001 -1.211333  0.505250 -1.483098
           000002 -0.361248  1.173286  0.240396
           000004 -0.968436 -1.293610 -2.134791
           000005 -0.634221  1.243630 -0.753569
           000006 -0.703299 -0.499816 -0.971731
...                     ...       ...       ...
2020-12-31 000001 -0.345097  1.993560  0.791049
           000002 -0.456381  0.286439 -0.227461
           000004 -0.543730 -1.505309 -1.089650
           000005 -0.397100 -1.362151 -0.438649
           000006 -0.715385 -0.167065 -0.936981

[1240 rows x 3 columns]
date        code  
2019-12-25  000001   -1.211333
            000002   -0.361248
            000004   -0.968436
            000005   -0.634221
            000006   -0.703299
                        ...   
2020-12-31  000001   -0.345097
            000002   -0.456381
            000004   -0.543730
            000005   -0.397100
            000006   -